In [ ]:
#Clone the transformers repo into the notebook
!git clone https://github.com/anur98/transformers

fatal: destination path 'transformers' already exists and is not an empty directory.


In [ ]:
# Clone should now be in the machine
!ls

eval_tmp.txt  sample_data     songs.txt      transformers
mymodel2      song_short.txt  train_tmp.txt


In [ ]:
!nvidia-smi

Tue May  3 15:32:00 2022       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 460.32.03    Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla P100-PCIE...  Off  | 00000000:00:04.0 Off |                    0 |
| N/A   35C    P0    27W / 250W |      0MiB / 16280MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [ ]:
import os
#os.chdir("transformers")
os.chdir("/content/transformers/examples/tensorflow/language-modeling")
!ls


eval_tmp.txt  requirements.txt	run_mlm.py
README.md     run_clm.py	train_tmp.txt


In [ ]:
!pip install -r requirements.txt

In [ ]:
!ls

eval_tmp.txt  requirements.txt	run_mlm.py
README.md     run_clm.py	train_tmp.txt


In [ ]:
!pip install pyarrow --upgrade

In [ ]:
!pwd


/content/transformers/examples/tensorflow/language-modeling


In [ ]:
!pip install git+https://github.com/anur98/transformers

  Cloning https://github.com/anur98/transformers to /tmp/pip-req-build-os4jvaaa
  Running command git clone -q https://github.com/anur98/transformers /tmp/pip-req-build-os4jvaaa
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
    Preparing wheel metadata ... done


In [ ]:

!pwd
%cd /content

/content/transformers/examples/tensorflow/language-modeling
/content


In [ ]:
"""
Now load the data line by line
"""
from sklearn.model_selection import train_test_split

with open('songs.txt', 'r') as data:
  dataset = ["<|title|>" + x.strip() for x in data.readlines()]

train, eval = train_test_split(dataset, train_size=.9, random_state=2020)
print("training size:" + str(len(train)))
print("Evaluation size: " + str(len(eval)))

with open('train_tmp.txt', 'w') as file_handle:
  file_handle.write("<|endoftext|>".join(train))

with open('eval_tmp.txt', 'w') as file_handle:
  file_handle.write("<|endoftext|>".join(eval))

training size:155205
Evaluation size: 17245


In [ ]:
%cd /content/transformers/examples/tensorflow/language-modeling
!cp "/content/train_tmp.txt" "/content/transformers/examples/tensorflow/language-modeling/"
!cp "/content/eval_tmp.txt" "/content/transformers/examples/tensorflow/language-modeling/"

/content/transformers/examples/tensorflow/language-modeling


In [ ]:
!python run_clm.py \
--model_type distilgpt2 \
--model_name_or_path distilgpt2 \
--train_file "train_tmp.txt" \
--do_train \
--validation_file "eval_tmp.txt" \
--do_eval \
--per_gpu_train_batch_size 1 \
--save_steps -1 \
--num_train_epochs 5 \
--fp16 \
--output_dir="/content/mymodel2"

Using custom data configuration default-cdb2460c1167ec09
Extracting data files: 100% 2/2 [00:00<00:00, 1363.56it/s]
Dataset text downloaded and prepared to /root/.cache/huggingface/datasets/text/default-cdb2460c1167ec09/0.0.0/4b86d314f7236db91f0a0f5cda32d4375445e64c5eda2692655dd99c2dac68e8. Subsequent calls will reuse this data.
100% 2/2 [00:00<00:00, 1112.10it/s]
loading configuration file https://huggingface.co/distilgpt2/resolve/main/config.json from cache at /root/.cache/huggingface/transformers/f985248d2791fcff97732e4ee263617adec1edb5429a2b8421734c6d14e39bee.422318838d1ec4e061efb4ea29671cb2a044e244dc69229682bebd7cacc81631
Model config GPT2Config {
  "_name_or_path": "distilgpt2",
  "_num_labels": 1,
  "activation_function": "gelu_new",
  "architectures": [
    "GPT2LMHeadModel"
  ],
  "attn_pdrop": 0.1,
  "bos_token_id": 50256,
  "embd_pdrop": 0.1,
  "eos_token_id": 50256,
  "id2label": {
    "0": "LABEL_0"
  },
  "initializer_range": 0.02,
  "label2id": {
    "LABEL_0": 0
  },
  

In [ ]:
from transformers import TFGPT2LMHeadModel
from transformers import GPT2Tokenizer
2
model = TFGPT2LMHeadModel.from_pretrained("/content/mymodel2/", from_pt=False)
tokenizer = GPT2Tokenizer.from_pretrained("gpt2")

All model checkpoint layers were used when initializing TFGPT2LMHeadModel.

All the layers of TFGPT2LMHeadModel were initialized from the model checkpoint at /content/mymodel2/.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFGPT2LMHeadModel for predictions without further training.


Downloading:   0%|          | 0.00/0.99M [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/446k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/665 [00:00<?, ?B/s]

In [ ]:
input_ids = tokenizer.encode("I love deep learning", return_tensors='tf')

In [ ]:
# the tf tensor object
input_ids[0]

<tf.Tensor: shape=(4,), dtype=int32, numpy=array([  40, 1842, 2769, 4673], dtype=int32)>

In [ ]:
generated_text_samples = model.generate(
    input_ids, 
    max_length=150,  
    num_return_sequences=5,
    no_repeat_ngram_size=2,
    repetition_penalty=1.5,
    top_p=0.92,
    temperature=.85,
    do_sample=True,
    top_k=125,
    early_stopping=True
)

Setting `pad_token_id` to 50256 (first `eos_token_id`) to generate sequence


In [ ]:
#Print output for each sequence generated above
for i, beam in enumerate(generated_text_samples):
  print("{}: {}".format(i,tokenizer.decode(beam, skip_special_tokens=True)))
  print()

0: I love deep learning

1: I love deep learning, it's a way to go

2: I love deep learning,

3: I love deep learning

4: I love deep learning



In [ ]:
!ls

eval_tmp.txt  requirements.txt	run_mlm.py
README.md     run_clm.py	train_tmp.txt
